In [1]:
import pandas as pd
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn import metrics
from collections import defaultdict
import joblib

import sys

import os

import time
start_time = time.time()


infile='/Users/irffanalahi/Research/Research_update/SoftRD/largerEXP/BL22genepromdelta.7/insilico/training/RF/BL22_genepromSM_0.7.txt_TRAININGbestref.txt_result_dupindex_binnedstats.pkl_totalformatteddata.pkl'

outfol=infile+"_model"

os.mkdir(outfol)

gtcol='label2'

n_trees=10000



indf=pd.read_pickle(infile)

indf.head()

,index,NaiveCD4-others,NaiveCD8-others,nB-others,NK-others,PC-others,Mono-others,M0-others,M1-others,M2-others,...,maxscore_beforeCpGweight,maxscoredCT,maxscore,deltabasedfragassignment,filename,label,Last,label2,deltaMedian,deltaMean
0,Eo.HWI-ST537:223:C4DVTACXX:7:1211:7435:87447,-0.181992,-0.146024,-0.188627,-0.131008,-0.087008,-0.121579,0.035913,0.019849,0.431214,...,0.769421,iDC-others,0.769421,iDC-others,BL22train_NR_3215798_insilmix7_sorted_binnedst...,Eo,HWI-ST537:223:C4DVTACXX:7:1211:7435:87447,NOTiDC,-0.103508,-5.046468e-18
1,Eo.HWI-ST537:224:C4E15ACXX:4:1214:13241:31111,-0.363675,0.077373,-0.556786,0.202040,-0.471230,-0.487643,-0.497421,-0.536532,-0.456214,...,1.430373,em8-others,1.430373,em8-others,BL22train_NR_3215798_insilmix7_sorted_binnedst...,Eo,HWI-ST537:224:C4E15ACXX:4:1214:13241:31111,NOTem8,-0.459706,-2.018587e-17
2,Eo.HWI-ST537:224:C4E15ACXX:4:2108:11851:51609,-0.078167,-0.018802,-0.051278,-0.062452,-0.053722,-0.026135,-0.065595,-0.078167,-0.053373,...,0.969452,PMN-others,0.969452,PMN-others,BL22train_NR_3215798_insilmix7_sorted_binnedst...,Eo,HWI-ST537:224:C4E15ACXX:4:2108:11851:51609,NOTPMN,-0.052325,-1.968123e-16
3,Eo.HWI-ST537:224:C4E15ACXX:5:2205:15200:28811,-0.238056,-0.233865,-0.196849,-0.222341,-0.203135,0.735881,0.502960,0.526008,0.567563,...,0.735881,Mono-others,0.735881,Mono-others,BL22train_NR_3215798_insilmix7_sorted_binnedst...,Eo,HWI-ST537:224:C4E15ACXX:5:2205:15200:28811,NOTMono,-0.199992,2.649396e-17
4,Eo.HWI-ST537:224:C4E15ACXX:6:1201:4904:83947,-0.182183,-0.131548,-0.225833,-0.133294,-0.138881,-0.130500,-0.180087,-0.080563,-0.141675,...,2.876167,Eo-others,2.876167,Eo-others,BL22train_NR_3215798_insilmix7_sorted_binnedst...,Eo,HWI-ST537:224:C4E15ACXX:6:1201:4904:83947,Eo,-0.146825,-1.709491e-16


In [2]:
#BL22
features=['NaiveCD4-others','NaiveCD8-others','nB-others','NK-others','PC-others','Mono-others','M0-others','M1-others','M2-others','iDC-others','mDC-others','PMN-others','cm8-others','em8-others','Eo-others','Tregs-others','em4-others','ed8-others','Mg-others','cm4-others','Er-others','mB-others','LENhypoCpG', 'LENhyperCpG', 'total_cpg', 'LENhypoCpG_BY_total_cpg','deltaMedian', 'deltaMean']


#BL14
#features=['CD4-others', 'CD8-others', 'nB-others', 'NK-others','Mn-others', 'mNeu-others', 'm8-others', 'DC-others', 'Eo-others','Tr-others', 'm4-others', 'Mg-others', 'Er-others', 'mB-others','LENhypoCpG', 'LENhyperCpG', 'total_cpg', 'LENhypoCpG_BY_total_cpg','deltaMedian', 'deltaMean']

In [3]:
#BL22

scorecolumns=['NaiveCD4-others','NaiveCD8-others','nB-others','NK-others','PC-others','Mono-others','M0-others','M1-others','M2-others','iDC-others','mDC-others','PMN-others','cm8-others','em8-others','Eo-others','Tregs-others','em4-others','ed8-others','Mg-others','cm4-others','Er-others','mB-others']




#BL14
#scorecolumns=['CD4-others', 'CD8-others', 'nB-others', 'NK-others','Mn-others', 'mNeu-others', 'm8-others', 'DC-others', 'Eo-others','Tr-others', 'm4-others', 'Mg-others', 'Er-others', 'mB-others']

In [4]:
for score in scorecolumns:
    
    tempct=score.replace('-others','')
    
 
  
    indfsingleCT=indf[(indf[gtcol]==tempct) | (indf[gtcol]=='NOT'+tempct)].copy()
    
    
    
    indfsingleCT.loc[indfsingleCT[gtcol]==tempct,gtcol]=1
    indfsingleCT.loc[indfsingleCT[gtcol]=='NOT'+tempct,gtcol]=0
    

    
    indfsingleCT[gtcol]=indfsingleCT[gtcol].astype(int)
    
    
    trainY=indfsingleCT[gtcol]
    trainX=indfsingleCT[features]
    
    
    
    
   
    
    
    RDF_classifier = RandomForestClassifier(oob_score=True,n_estimators=n_trees)
    
    RDF_classifier.fit(trainX, trainY)
    
    joblib.dump(RDF_classifier, outfol+"/"+tempct+".pkl")

In [5]:
end_time = time.time()

time_elapsed = (end_time - start_time)

print(time_elapsed)

439.50795698165894
